In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

import os
import sys
sys.path.append('..')

import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
from tensorflow.python.training import saver as saver_lib

from src.data.cifar100 import CIFAR_100_CLASSES, load_cifar100_data
from src.meta.tensor_apis import TensorApi
from src.meta.meta import MetaModel, MetaModelFactory
from src.utils.debug_utils import print_list
from src.utils.file_utils import MODELS_DIR
from src.utils.logger import HuliLogging

print(tf.__version__)

/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

1.14.0


/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([

In [2]:
logger = HuliLogging.get_logger(__name__)
HuliLogging.attach_stdout()

In [3]:
%%bash

find ../models -type f |                     grep -v basic | grep -v batchn_000 | grep -v conv | grep -v resnet | grep -v .gitignore | grep -v h5 | xargs rm
find ../models -type f | cut -c 11- | sort | grep -v basic | grep -v batchn_000 | grep -v conv | grep -v resnet | grep -v .gitignore

batchn_001.h5


In [4]:

# https://github.com/tensorflow/tensorflow/issues/20532
def freeze_graph_from_issues(model_dir, output_tensors, output_pb):
    """ Freeze graph model into **.pb** file
    
    Extract the sub graph defined by the output nodes and convert all its variables to constants

    Args:    
        ``model_dir`` (str): directory that contains all checkpoint files (.ckpt, .meta, .info)
        
        ``output_tensors`` (str):  comma separated list of all the output node's names

        ``output_pb`` (str): output **.pb** frozen graph file      
        
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    if not output_tensors:
        tf.loogin.error("You need to supply the name of a node to --output_tensors.")
        return -1

    # Retrieve checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # Clear original devices from graph
    clear_devices = True

    with tf.Session(graph=tf.Graph()) as sess:
        # Import graph from .meta file
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices=clear_devices)

        input_graph_def = tf.get_default_graph().as_graph_def()

        # and restore weights
        saver.restore(sess, input_checkpoint)

        # Convert variables to constants
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            input_graph_def,            
            output_tensors.split(","),
            variable_names_blacklist=['global_step']
        )

        # Dump frozen model
        with tf.gfile.GFile(output_pb, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        tf.logging.info("%d ops in the final graph." % len(output_graph_def.node))

In [5]:
_NAME = 'batchn'
_EPOCH = 1

<tf.Tensor 'keras_learning_phase:0' shape=() dtype=bool>

In [6]:
K.set_learning_phase(0)
ret, m = MetaModelFactory.from_h5(_NAME, _EPOCH)
km = m.delegate.keras_model

filename = '%s_%03d' % (_NAME, _EPOCH)

print('in_names:',  [out.op.name for out in km.inputs])
out_names = [out.op.name for out in km.outputs]
out_names = ','.join(out_names)
print('out_names:', out_names)


sess = K.get_session( )
print('Saving checkpoints...')

saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)
checkpoint_path = saver.save(sess, os.path.join(MODELS_DIR, filename + '_saved_ckpt'),
                             global_step=0, latest_filename='checkpoint')
print('checkpoint_path: ', checkpoint_path)
print('exists:', os.path.exists(checkpoint_path))
checkpoint_state = os.path.join(MODELS_DIR, 'checkpoint')
print('checkpoint_state: ', checkpoint_state)
print('exists:', os.path.exists(checkpoint_state))
out_pb = os.path.join(MODELS_DIR, filename + '.pb')
print('Freezing to', out_pb, '...')
freeze_graph_from_issues(MODELS_DIR, out_names, out_pb)

W0808 15:10:37.190644 140506672251008 deprecation_wrapper.py:119] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:153: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



2019-08-08 15:10:37,191 DEBUG [src.meta.keras:216] (MainThread) batchn Loading keras model from /space/code/deep-learning/models/batchn_001.h5...


W0808 15:10:37.198183 140506672251008 deprecation.py:506] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0808 15:10:37.199352 140506672251008 deprecation.py:506] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0808 15:10:37.202030 140506672251008 deprecation.py:506] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning/lib/python3.6/site-packages/tensor

in_names: ['batchn0']
out_names: fc100/Softmax
Saving checkpoints...


FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Error while reading resource variable Adam/decay from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/Adam/decay)
	 [[node Adam/decay/Read/ReadVariableOp (defined at ../src/meta/keras.py:218) ]]
	 [[fc100/kernel/Read/ReadVariableOp/_37]]
  (1) Failed precondition: Error while reading resource variable Adam/decay from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/Adam/decay)
	 [[node Adam/decay/Read/ReadVariableOp (defined at ../src/meta/keras.py:218) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Adam/decay/Read/ReadVariableOp':
  File "/home/maka/.pyenv/versions/3.6.8/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/

In [ ]:
%%bash

find ../models -type f | cut -c 11- | sort | grep -v basic | grep -v batchn_000 | grep -v conv | grep -v resnet | grep -v .gitignore

In [18]:
km.save('opt.h5', include_optimizer=True)

In [19]:
km.save('no_opt.h5', include_optimizer=False)

In [20]:
km.save_weights('weights.h5')

In [21]:
%%bash
ls -l | grep h5 | sort

-rw-rw-r-- 1 maka maka 1244280 Aug  8 15:29 weights.h5
-rw-rw-r-- 1 maka maka 1245616 Aug  8 15:29 no_opt.h5
-rw-rw-r-- 1 maka maka 3714448 Aug  8 15:29 opt.h5
